In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import catboost as cb
from catboost import CatBoostClassifier
import time

from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, f1_score, roc_auc_score, average_precision_score, precision_score, recall_score, classification_report
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split

from cc_preprocessor import Preprocessor

np.random.seed(42)

In [2]:
df = pd.read_csv('data/Training_TriGuard.csv')
df = df.dropna(subset=['subrogation'])

In [3]:
pre = Preprocessor(smoothing_factor=5, mode = 'catboost')

In [4]:
X = df.drop(columns=["subrogation"]).copy()
y = df["subrogation"].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=0)

In [5]:
y_train.value_counts(normalize=True)

subrogation
0.0    0.77141
1.0    0.22859
Name: proportion, dtype: float64

In [6]:
y_test.value_counts(normalize=True)

subrogation
0.0    0.771296
1.0    0.228704
Name: proportion, dtype: float64

In [7]:
pre.fit(X_train, y_train)

X_train_proc = pre.transform(X_train)
X_test_proc = pre.transform(X_test)

X_test_proc = X_test_proc.reindex(columns=X_train_proc.columns, fill_value=0)

Fitting Preprocessor in 'catboost' mode...
CatBoost mode: Skipping target encoding learning.
Fit complete.
Transforming data in 'catboost' mode...
CatBoost mode: Skipping target encoding application.
CatBoost mode: Dropping unused object/datetime columns...
Dropping: ['witness_present_ind', 'claim_date']
Transform complete.
Transforming data in 'catboost' mode...
CatBoost mode: Skipping target encoding application.
CatBoost mode: Dropping unused object/datetime columns...
Dropping: ['witness_present_ind', 'claim_date']
Transform complete.


## Vanilla CatBoost Model (Default Parameters)

In [8]:
cb_clf = cb.CatBoostClassifier(
    objective='Logloss',
    random_state=42,
    thread_count=-1
)

In [9]:
cb_clf.fit(X_train_proc, y_train, verbose=False)

In [11]:
test_probabilities = cb_clf.predict_proba(X_test_proc)[:, 1]

test_classes = cb_clf.predict(X_test_proc)

print(f"Accuracy: {accuracy_score(y_test, test_classes)}")
print(f"F1 Score: {f1_score(y_test, test_classes)}")
print(f"ROC AUC Score: {roc_auc_score(y_test, test_probabilities)}") # Use probabilities
print(f"PR AUC (Average Precision): {average_precision_score(y_test, test_probabilities)}") # Use probabilities
print(f"Precision: {precision_score(y_test, test_classes)}")
print(f"Recall: {recall_score(y_test, test_classes)}")

Accuracy: 0.8138888888888889
F1 Score: 0.5175228036485837
ROC AUC Score: 0.8358050264640269
PR AUC (Average Precision): 0.5998511662785648
Precision: 0.6356132075471698
Recall: 0.43643724696356273


## CatBoost with Randomized Search for Hyperparameter Tuning

For `roc_auc` scoring

In [14]:
print("Starting randomized search tuning...")
start_time = time.time()

# Define a larger parameter space
param_dist = {
    'depth': [3, 4, 5], 
    'learning_rate': [0.0025, 0.005, 0.0075], 
    'n_estimators': [2000, 2250, 2500],
    'subsample': [0.4, 0.5, 0.6],
    'rsm': [0.8, 0.9, 1.0],
    'l2_leaf_reg': [8, 9, 10]
}

# Use Randomized Search
random_search = RandomizedSearchCV(
    estimator=cb_clf,
    param_distributions=param_dist,
    n_iter=150,
    cv=3,
    scoring='roc_auc',
    random_state=42,
    n_jobs=-1,
    verbose=1
)

random_search.fit(X_train_proc, y_train, verbose=False)

random_search_time = time.time() - start_time
print(f"Randomized search complete. Time taken: {random_search_time:.2f} seconds")

# Output the best parameters
print("\nBest parameters:")
for param, value in random_search.best_params_.items():
    print(f"  {param}: {value}")

print(f"\nBest cross-validation accuracy: {random_search.best_score_:.4f}")

Starting randomized search tuning...
Fitting 3 folds for each of 150 candidates, totalling 450 fits
Randomized search complete. Time taken: 774.65 seconds

Best parameters:
  subsample: 0.5
  rsm: 0.9
  n_estimators: 2000
  learning_rate: 0.005
  l2_leaf_reg: 9
  depth: 4

Best cross-validation accuracy: 0.8407


In [15]:
best_model = random_search.best_estimator_
test_predictions_gs = best_model.predict(X_test_proc)
round_test_predictions_gs = [round(p) for p in test_predictions_gs]
print(f"Accuracy: {accuracy_score(y_test, round_test_predictions_gs)}")
print(f"F1 Score: {f1_score(y_test, round_test_predictions_gs)}")
print(f"ROC AUC Score: {roc_auc_score(y_test, test_predictions_gs)}") # Use probabilities
print(f"PR AUC (Average Precision): {average_precision_score(y_test, test_predictions_gs)}") # Use probabilities
print(f"Precision: {precision_score(y_test, round_test_predictions_gs)}")
print(f"Recall: {recall_score(y_test, round_test_predictions_gs)}")

Accuracy: 0.8172222222222222
F1 Score: 0.5192401363857769
ROC AUC Score: 0.6815757882100208
PR AUC (Average Precision): 0.4112121992021619
Precision: 0.6515892420537898
Recall: 0.43157894736842106


for `f1` scoring

In [16]:
print("Starting randomized search tuning...")
start_time = time.time()

# Define a larger parameter space
param_dist_2 = {
    'learning_rate': [0.015, 0.02, 0.03],
    'l2_leaf_reg': [13, 15, 17],
    'subsample': [0.3, 0.4, 0.5],
    'rsm': [0.7, 0.8, 0.9],
    'n_estimators': [1750, 2000, 2250],
    'depth': [2, 3, 4]
}

# Use Randomized Search
random_search_2 = RandomizedSearchCV(
    estimator=cb_clf,
    param_distributions=param_dist_2,
    n_iter=150,
    cv=3,
    scoring='f1',
    random_state=42,
    n_jobs=-1,
    verbose=1
)

random_search_2.fit(X_train_proc, y_train, verbose=False)

random_search_time = time.time() - start_time
print(f"Randomized search complete. Time taken: {random_search_time:.2f} seconds")

# Output the best parameters
print("\nBest parameters:")
for param, value in random_search_2.best_params_.items():
    print(f"  {param}: {value}")

print(f"\nBest cross-validation accuracy: {random_search_2.best_score_:.4f}")

Starting randomized search tuning...
Fitting 3 folds for each of 150 candidates, totalling 450 fits
Randomized search complete. Time taken: 468.98 seconds

Best parameters:
  subsample: 0.3
  rsm: 0.8
  n_estimators: 2000
  learning_rate: 0.02
  l2_leaf_reg: 13
  depth: 2

Best cross-validation accuracy: 0.5147


In [17]:
best_model_2 = random_search_2.best_estimator_
test_predictions_gs_2 = best_model_2.predict(X_test_proc)
round_test_predictions_gs_2 = [round(p) for p in test_predictions_gs_2]
print(f"Accuracy: {accuracy_score(y_test, round_test_predictions_gs_2)}")
print(f"F1 Score: {f1_score(y_test, round_test_predictions_gs_2)}")
print(f"ROC AUC Score: {roc_auc_score(y_test, test_predictions_gs_2)}") # Use probabilities
print(f"PR AUC (Average Precision): {average_precision_score(y_test, test_predictions_gs_2)}") # Use probabilities
print(f"Precision: {precision_score(y_test, round_test_predictions_gs_2)}")
print(f"Recall: {recall_score(y_test, round_test_predictions_gs_2)}")

Accuracy: 0.8183333333333334
F1 Score: 0.5276841598459316
ROC AUC Score: 0.6865682305310788
PR AUC (Average Precision): 0.4160121671191805
Precision: 0.6508313539192399
Recall: 0.4437246963562753


## CatBoost with Optuna Tuning

In [9]:
import optuna
from optuna.integration import CatBoostPruningCallback

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
CAT_FEATURES = pre.cat_for_encoding_
print(CAT_FEATURES)

['accident_site', 'accident_type', 'channel', 'vehicle_category', 'vehicle_color', 'living_status', 'claim_day_of_week', 'gender', 'in_network_bodyshop', 'season']


In [23]:
def objective(trial: optuna.trial.Trial) -> float:

    params = {
        'iterations': 1000,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'depth': trial.suggest_int('depth', 3, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 10.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0, step=0.1),
        'random_strength': trial.suggest_float('random_strength', 1e-8, 1.0, log=True), 
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1.0, 10.0),
        
        'eval_metric': 'F1',
        'task_type': 'CPU',
        'verbose': False,
        'early_stopping_rounds': 100
    }

    params['eval_metric'] = 'Logloss'
    
    model = CatBoostClassifier(**params)
    
    model.fit(
        X_train_proc, y_train,
        eval_set=(X_test_proc, y_test),
        cat_features=CAT_FEATURES,
        verbose=False
    )

    y_preds = model.predict(X_test_proc)
    
    manual_f1_score = f1_score(y_test, y_preds, pos_label=1)
    
    return manual_f1_score

In [24]:
print("\n2. Starting Optuna study...")

study = optuna.create_study(
    direction='maximize',
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=10)
)

study.optimize(
    objective, 
    n_trials=50, # Number of trials to run
    show_progress_bar=True
)

print("\n" + "="*50)
print("Optuna study finished.")
print(f"Number of finished trials: {len(study.trials)}")

print("\nBest trial:")
best_trial = study.best_trial
    
print(f"  Value (Max F1 Score): {best_trial.value:.4f}") # <-- CHANGED comment
    
print("  Best Hyperparameters:")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")

[I 2025-11-05 18:18:18,338] A new study created in memory with name: no-name-084a018a-9d5f-41b5-a286-4f63f98786cf



2. Starting Optuna study...


Best trial: 0. Best value: 0.561864:   2%|▏         | 1/50 [00:05<04:13,  5.18s/it]

[I 2025-11-05 18:18:23,522] Trial 0 finished with value: 0.5618639528655597 and parameters: {'learning_rate': 0.02551511688062508, 'depth': 9, 'l2_leaf_reg': 0.5585952194165865, 'subsample': 0.9, 'random_strength': 0.05678070706319962, 'bagging_temperature': 0.3111753154449576, 'border_count': 77, 'scale_pos_weight': 5.51116918821126}. Best is trial 0 with value: 0.5618639528655597.


Best trial: 0. Best value: 0.561864:   4%|▍         | 2/50 [00:08<03:20,  4.18s/it]

...

[I 2025-11-05 18:20:49,988] Trial 47 finished with value: 0.5718341851220897 and parameters: {'learning_rate': 0.013056394787128346, 'depth': 3, 'l2_leaf_reg': 0.016718538527243997, 'subsample': 1.0, 'random_strength': 2.5121221122812446e-07, 'bagging_temperature': 0.8274962419585674, 'border_count': 216, 'scale_pos_weight': 3.9554068822068076}. Best is trial 25 with value: 0.5980662983425414.


Best trial: 25. Best value: 0.598066:  98%|█████████▊| 49/50 [02:36<00:03,  3.76s/it]

[I 2025-11-05 18:20:55,150] Trial 48 finished with value: 0.5362657931679925 and parameters: {'learning_rate': 0.021791500345252613, 'depth': 4, 'l2_leaf_reg': 0.15633868811149645, 'subsample': 0.8, 'random_strength': 7.462469482202366e-08, 'bagging_temperature': 0.0910785490242269, 'border_count': 248, 'scale_pos_weight': 1.0590838000094103}. Best is trial 25 with value: 0.5980662983425414.


Best trial: 25. Best value: 0.598066: 100%|██████████| 50/50 [02:38<00:00,  3.17s/it]

[I 2025-11-05 18:20:56,860] Trial 49 finished with value: 0.5887913571910871 and parameters: {'learning_rate': 0.07049302502086392, 'depth': 5, 'l2_leaf_reg': 0.04009605589203643, 'subsample': 1.0, 'random_strength': 0.0010171754777477755, 'bagging_temperature': 0.9251194820184121, 'border_count': 194, 'scale_pos_weight': 2.4951009872284664}. Best is trial 25 with value: 0.5980662983425414.

Optuna study finished.
Number of finished trials: 50

Best trial:
  Value (Max F1 Score): 0.5981
  Best Hyperparameters:
    learning_rate: 0.04562050017200123
    depth: 5
    l2_leaf_reg: 0.04435233918938249
    subsample: 0.9
    random_strength: 1.4159755470392207e-08
    bagging_temperature: 0.8472034124124315
    border_count: 224
    scale_pos_weight: 2.322403530420936


In [30]:
best_params = study.best_trial.params
print(best_params)

final_params = best_params.copy()
final_params.update({
    'iterations': 2000, # Use more iterations for the final model
    'eval_metric': 'Logloss', # Use Logloss for training/stopping
    'task_type': 'CPU',
    'early_stopping_rounds': 50 # Keep early stopping
})

best_model = CatBoostClassifier(**final_params)

best_model.fit(
    X_train_proc, y_train,
    eval_set=(X_test_proc, y_test),
    cat_features=CAT_FEATURES,
    verbose=False
)

print(f"\nFinal Model Score (from best Logloss iteration):")
y_preds_final = best_model.predict(X_test_proc)
final_f1 = f1_score(y_test, y_preds_final, pos_label=1)
print(f"  Manual F1:class=1 Score: {final_f1:.4f}")
        
print("\n  Full Classification Report:")
print(classification_report(y_test, y_preds_final, target_names=['Class 0.0', 'Class 1.0']))

{'learning_rate': 0.04562050017200123, 'depth': 5, 'l2_leaf_reg': 0.04435233918938249, 'subsample': 0.9, 'random_strength': 1.4159755470392207e-08, 'bagging_temperature': 0.8472034124124315, 'border_count': 224, 'scale_pos_weight': 2.322403530420936}

Final Model Score (from best Logloss iteration):
  Manual F1:class=1 Score: 0.5981

  Full Classification Report:
              precision    recall  f1-score   support

   Class 0.0       0.90      0.81      0.85      4165
   Class 1.0       0.52      0.70      0.60      1235

    accuracy                           0.78      5400
   macro avg       0.71      0.76      0.73      5400
weighted avg       0.81      0.78      0.79      5400



In [31]:
# Output module, from model_citizens.ipynb
real_test = pd.read_csv("data/Testing_TriGuard.csv")

X_real_test_proc = pre.transform(real_test)
X_real_test_proc = X_real_test_proc.reindex(columns=X_train_proc.columns, fill_value=0)
real_pred_proba = best_model.predict_proba(X_real_test_proc)[:, 1]
real_pred_label = (real_pred_proba >= 0.5).astype(int)

prediction = pd.DataFrame({
    "claim_number": real_test["claim_number"],
    "subrogation": real_pred_label
})

print(prediction.head())

Transforming data in 'catboost' mode...
CatBoost mode: Skipping target encoding application.
CatBoost mode: Dropping unused object/datetime columns...
Dropping: ['witness_present_ind', 'claim_date']
Transform complete.
   claim_number  subrogation
0       3126034            0
1       7380142            1
2       4655051            0
3       6728725            1
4       9848460            1


In [33]:
prediction.to_csv("results/catboost_prediction.csv", index=False)